# Lecture 8: Introduction to Pytorch & Neural Networks

**Jonny, June 2024**

## Lecture Learning Objectives
<hr>

- Describe the difference between `numpy` and `torch` arrays (`np.array` vs. `torch.Tensor`)
- Explain fundamental concepts of neural networks such as layers, nodes, activation functions, etc.
- Create a simple neural network in PyTorch for regression or classification

## Imports
<hr>

In [ ]:
import sys
import numpy as np
import pandas as pd
import torch
from torchsummary import summary
from torch import nn, optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.datasets import make_regression, make_circles, make_blobs
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from utils.plotting import *

## 1. Introduction
<hr>

- PyTorch is a Python-based tool for scientific computing that provides several main features:
    - `torch.Tensor`, an n-dimensional array similar to that of `numpy`, but which can run on GPUs
    - Computational graphs for building neural networks
    - Automatic differentiation for training neural networks (more on this next lecture)
- You can install PyTorch from: https://pytorch.org/

## 2. PyTorch's Tensor
<hr>

- In PyTorch a tensor is just like NumPy's `ndarray` that we have become so familiar with.
- A key difference between PyTorch's `torch.Tensor` and numpy's `np.array` is that `torch.Tensor` was constructed to integrate with GPUs and PyTorch's computational graphs (more on that next lecture though)

### 2.1. `ndarray` vs `tensor`

- Creating and working with tensors is much the same as with numpy `ndarrays`
- You can create a tensor with `torch.tensor()`:

In [ ]:
tensor_1 = torch.tensor([1, 2, 3])
tensor_2 = torch.tensor([1, 2, 3], dtype=torch.float32)
tensor_3 = torch.tensor(np.array([1, 2, 3]))

for t in [tensor_1, tensor_2, tensor_3]:
    print(f"{t}, dtype: {t.dtype}")

- PyTorch also comes with most of the `numpy` functions we're already familiar with:

In [ ]:
 # zeroes

In [ ]:
  # ones

In [ ]:
  # random normal

In [ ]:
# rand uniform

- Just like in NumPy we can look at the shape of a tensor with the `.shape` attribute:

### 2.2. Tensors and Data Types

- Different dtypes have different memory and computational implications (see the end of [Lecture 1](https://pages.github.ubc.ca/MDS-2020-21/DSCI_572_sup-learn-2_students/lectures/lecture1_floating-point.html#precision-vs-memory-vs-speed) or [Lecture 5 of DSCI 511](https://pages.github.ubc.ca/MDS-2020-21/DSCI_511_py-prog_students/lectures/lecture5-numpy-addendum.html) for more)
- In Pytorch we'll be building networks that require thousands or even millions of floating point calculations!
- In such cases, using a smaller dtype like `float32` can significantly speed up computations and reduce memory requirements
- The default float dtype in pytorch `float32`, as opposed to numpy's `float64`
- In fact some operations in Pytorch will even throw an error if you pass a high-memory `dtype`!

- But just like in numpy, you can always specify the particular dtype you want using the `dtype` argument:

### 2.3. Operations on Tensors

- Tensors operate just like `ndarrays` and have a variety of familiar methods that can be called off them:

In [ ]:
 # broadcasting betweean a 1 x 3 and 3 x 1 tensor

### 2.4. Indexing

- Once again, same as NumPy!

In [ ]:
p

### 2.5. Numpy Bridge

- Sometimes we might want to convert a tensor back to a NumPy array
- We can do that using the `.numpy()` method

### 2.6. GPU and CUDA Tensors

- GPU is a graphical processing unit (as opposed to a CPU: central processing unit)
- GPUs were originally developed for gaming, they are very fast at performing operations on large amounts of data by performing them in parallel (think about updating the value of all pixels on a screen very quickly as a player moves around in a game)
- More recently, GPUs have been adapted for more general purpose programming
- Neural networks can typically be broken into smaller computations that can be performed in parallel on a GPU
- PyTorch is tightly integrated with CUDA - a software layer that facilitates interactions with a GPU (if you have one)
- You can check if you have GPU capability using:

In [ ]:
torch.cuda.is_available()  # my MacBook Pro does not have a GPU

- When training on a machine that has a GPU, you need to tell PyTorch you want to use it
- You'll see the following at the top of most PyTorch code:

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

- You can then use the `device` argument when creating tensors to specify whether you wish to use a CPU or GPU
- Or if you want to move a tensor between the CPU and GPU, you can use the `.to()` method:

In [ ]:
X = torch.rand(2, 2, 2, device=device)
print(X.device)

In [ ]:
# X.to('cuda')  # this would give an error for me so I'm commenting out

- We'll revisit GPUs later in the course when we are working with bigger datasets and more complex networks
- For now, we can work on the CPU just fine

## 3. Neural Network Basics
<hr>

- You've already learned about several machine learning algorithms (kNN, Random Forest, SVM, etc.)
- Neural networks are simply another algorithm and actually one of the simplest in my opinion!
- As we'll see, a neural network is just a sequence of linear and non-linear transformations
- Often you see something like this when learning about/using neural networks:

![](img/nn-6.png)

- So what on Earth does that all mean? Well we are going to build up some intuition one step at a time

### 3.1. Simple Linear Regression with a Neural Network

- Let's create a simple regression dataset with 500 observations:

- We know how to fit a simple linear regression to this data using sklearn:

- Here are the parameters of that fitted line:

- As an equation, that looks like this:

$$\hat{y}=-0.77 + 45.50X$$

- Or in matrix form:

$$\begin{bmatrix} \hat{y_1} \\ \hat{y_2} \\ \vdots \\ \hat{y_n} \end{bmatrix}=\begin{bmatrix} 1 & x_1 \\ 1 & x_2 \\ \vdots & \vdots \\ 1 & x_n \end{bmatrix} \begin{bmatrix} -0.77 \\ 45.55 \end{bmatrix}$$

- Or in graph form I'll represent it like this: 

![](img/nn-1.png)

### 3.2. Linear Regression with a Neural Network in PyTorch

- So let's implement the above in PyTorch to start gaining an intuition about neural networks!
- Every neural network model you build in PyTorch will inherit from `torch.nn.Module`
- Remember [class inheritance from DSCI 511](https://www.tomasbeuzen.com/python-programming-for-data-science/lectures/lecture3-tests-classes.html#inheritance-subclasses)? Inheritance allows us to inherit commonly needed functionality without having to write code ourselves!

>We'll explore `torch.nn.Module` more in the lab but if it helps, think about sklearn models: they all inherit common methods like `.fit()`, `.predict()`, `.score()`, etc. When creating a neural network, we define our own architecture but still want common functionality which we inherit from `torch.nn.Module`.

- Let's create a model called `linearRegression` and then I'll talk you through the syntax:

Let's step through the above:

```python
class linearRegression(nn.Module):
    def __init__(self, input_size, output_size):
        super().__init__() 
```

^ Here we're creating a class called `linearRegression` and inheriting the methods and attributes of `nn.Module` (hint: try typing `help(linearRegression)` to see all the things we inheritied from `nn.Module`).

```python
        self.linear = nn.Linear(input_size, output_size)
```

^ Here we're defining a "Linear" layer, which just means `wX + b`, i.e., the weights of the network, multiplied by the input features plus the bias.

```python
    def forward(self, x):
        out = self.linear(x)
        return out
```

^ PyTorch networks created with `nn.Module` must have a `forward()` method. It accepts the input data `x` and passes it through the defined operations. In this case, we are passing `x` into our linear layer and getting an output `out`.

- After defining the model class, we can create an instance of that class:

![](img/nn-2.png)

- We can check out our model using `print()`:

- Or the more useful `summary()` (which we imported at the top of this notebook with `from torchsummary import summary`):

- Notice how we have two parameters? We have one for the weight (`w1`) and one for the bias (`w0`)
- These were initialized randomly by PyTorch when we created our model. They can be accessed with `model.state_dict()`:

- Okay, before we move on, our `x` and `y` data are currently numpy arrays but they need to be PyTorch tensors
- Let's convert them:

- We have a working model right now and could tell it to give us some output with this syntax:

- Our prediction is pretty bad because our model is not trained/fitted yet!
- As we learned in the past few lectures, to fit our model we need:
    1. **a loss function** (called "criterion" in PyTorch) to tell us how good/bad our predictions are - we'll use mean squared error, `torch.nn.MSELoss()`
    2. **an optimization algorithm** to help optimise model parameters - we'll use GD, `torch.optim.SGD()`

- Before we train I'm going to create a "data loader" to help batch my data
- We'll talk more about these next lecture and in lab but they are just generators that yield data to us on request ([remember generators from 511?](https://pages.github.ubc.ca/MDS-2020-21/DSCI_511_py-prog_students/lectures/lecture2-loops-functions.html#generators))
- We'll use a `BATCH_SIZE = 50` (which should give us 10 batches because we have 500 data points)

- We should have 10 batches:

- We can look at a batch using this syntax:

- With those, let's train our simple network for 5 epochs of SGD!
> I'll explain all the code here next lecture but scan throught it, it's not too hard to see what's going on!

- Now our model has been trained, our parameters should be different than before:

- Comparing to our sklearn model, we get the same answer:

- We got pretty close! We could do better by changing the number of epochs or the learning rate
- So here is our simple network once again:

![](img/nn-2.png)

- By the way, check out what happens if we run `trainer()` again:

- Our model continues where we left off!
- This may or may not be what you want. We can start from scratch by re-making our `model` and `optimizer`.

### 3.3. Multiple Linear Regression with a Neural Network

- Okay, let's do a multiple linear regression now with 3 features
- So our network will look like this:

![](img/nn-3.png)

- Let's go ahead and create some data:

In [ ]:
# Create dataset
X, y = make_regression(n_samples=500, n_features=3, random_state=0, noise=10.0)
X_t = torch.tensor(X, dtype=torch.float32)
y_t = torch.tensor(y, dtype=torch.float32)
# Create dataloader
dataset = TensorDataset(X_t, y_t)
dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)

- And let's create the above model:

In [ ]:
model = linearRegression(input_size=3, output_size=1)

- We should now have 4 parameters (3 weights and 1 bias)

In [ ]:
summary(model, (3,));

- Looks good to me! Let's train the model and then compare it to sklearn's `LinearRegression()`:

In [ ]:
criterion = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=LEARNING_RATE)
trainer(model, criterion, optimizer, dataloader, epochs=5, verbose=True)

In [ ]:
sk_model = LinearRegression().fit(X, y)
pd.DataFrame({"w0": [sk_model.intercept_, model.state_dict()['linear.bias'].item()],
              "w1": [sk_model.coef_[0], model.state_dict()['linear.weight'][0, 0].item()],
              "w2": [sk_model.coef_[1], model.state_dict()['linear.weight'][0, 1].item()],
              "w3": [sk_model.coef_[2], model.state_dict()['linear.weight'][0, 2].item()]},
             index=['sklearn', 'pytorch']).round(2)

### 3.4. Non-linear Regression with a Neural Network

- Okay so we can make a simple network to imitate simple and multiple *linear* regression
- You're probably thinking, so what? But we're getting to the good stuff I promise!
- For example, what happens when we have more complicated datasets like this?

In [ ]:
# Create dataset

# Create dataloader


- This is obviously non-linear, and we need to introduce some **non-linearities** into our network
- These non-linearities are what make neural networks so powerful and they are called **"activation functions"**
- We are going to create a new model class that includes a non-linearity - a sigmoid function:

$$S(X)=\frac{1}{1+e^{-x}}$$

- We'll talk more about activation functions later, but note how the sigmoid function non-linearly maps `x` to a value between 0 and 1
- Okay, so let's create the following network:

![](img/nn-5.png)

- All this means is that the value of each node in the hidden layer will be transformed by the "activation function", thus introducing non-linear elements to our model!
- There's two main ways of creating the above model in PyTorch, I'll show you both:

- Note how our `forward()` method now passes `x` through the `nn.Sigmoid()` function after the hidden layer
- The above method is very clear and flexible, but I prefer using `nn.Sequential()` to combine my layers together in the constructor:

- Let's make an instance of our new class and confirm it has 10 parameters (6 weights + 4 biases):

- Okay, let's train:

- Take a look at those non-linear predictions! Cool!
- Our model is not great and we could make it better soon by adjusting the learning rate, the number of nodes, and the number of epochs
- But I really want you to see how **each of our hidden nodes is "engineering a non-linear feature"** to be used for the predictions, check it out:

### 3.5. Deep Learning

- You've probably heard the magic term "deep learning" and you're about to find out what it means!
- Really, it's just a neural network with more than 1 hidden layer! Easy!
- I like to think of each layer as a "feature engineerer", it is trying to extract the maximum amount of information from the layer before it
- There are arguments of "deep network" (many layers) vs "wide network" (many nodes), but for big datasets, "deep networks" have been shown to be very effective in practice
- Let's create a "deep" network of 2 layers:

![](img/nn-6.png)

- But just because the network is deeper, doesn't always mean it's better!

## 4. Activation Functions
<hr>

- As we learned above, activation functions are what allow us to model complex, non-linear functions
- There are **many** different activations functions:

- Activation functions should be non-linear and tend to be monotonic and continuously differentiable (smooth)
- But as you can see with the ReLU function above, that's not always the case!
- I wanted to point this out because it highlights how much of an art deep learning really is.
- Here's a great quote from [Yoshua Bengio](https://en.wikipedia.org/wiki/Yoshua_Bengio) (famous for his work in AI and deep learning) on his group experimenting with ReLU:

>"...one of the biggest mistakes I made was to think, like everyone else in the 90s, that you needed smooth non-linearities in order for backpropagation to work. because I thought that if we had something like rectifying nonlinearities, where you have a flat part, that it would be really hard to train, because the derivative would be zero in so many places. And when we started experimenting with ReLU, with deep nets around 2010, I was obsessed with the idea that, we should be careful about whether neurons won't saturate too much on the zero part. **But in the end, it turned out that, actually, the ReLU was working a lot better than the sigmoids and tanh, and that was a big surprise**...it turned out to work better, whereas I thought it would be harder to train!"

- Anyway, TL;DR **ReLU is probably the most popular these days**, but you can treat activation functions as hyperparams (to be optimized)

## 5. Neural Network Classification
<hr>

### 5.1. Binary Classification

- This will actually be the easiest part of the lecture
- Up until now, we've been looking at developing networks for regression tasks, but what if we want to do binary classification?
- Well, what did we do in Logistic Regression? We just passed the output of a regression into the Sigmoid Function to get a value between 0 and 1 (a probability of an observation belonging to the positive class) - we'll do the same thing here!
- Let's create a toy dataset:

- Let's create this network to model that dataset:

![](img/nn-7.png)

- I'm going to start using `ReLU` as our activation function(s) and `Adam` as our optimizer because these are what are currently, commonly used in practice.
- We are doing classificatio now so we'll need to use log loss (binary cross entropy) as our loss function:

$$f(w) = \sum_{x,y \in D} -y log(\hat{y}) - (1-y)log(1-\hat{y})$$

- In PyTorch, binary cross entropy loss criterion is `torch.nn.BCELoss`
- The formula expects a "probability" which is why we add a Sigmoid function to the end of out network.

- **BUT WAIT!**
- While we can do the above and then train with a `torch.nn.BCELoss` loss function, there's a better way!
- We can omit the Sigmoid function and just use `torch.nn.BCEWithLogitsLoss` (which combines a Sigmoid layer and the BCELoss)
- Why would we do this? It's numerically stable! (Did you do the log-sum-exp question in Lab 1? We use it here for stability!)
- From the docs:
>"*This version is more numerically stable than using a plain Sigmoid followed by a BCELoss as, by combining the operations into one layer, we take advantage of the log-sum-exp trick for numerical stability.*"
- So actually, here's our model (no Sigmoid layer at the end because it's included in the loss function we'll use):

- Let's train the model:

- To be clear, our model is just outputting some number between -∞ and +∞ (we aren't applying Sigmoid), so:
    - To get the probabilities we would need to pass them through a Sigmoid;
    - To get classes, we can apply some threshold (usually 0.5)
- For example, we would expect the point (0,0) to have a high probability and the point (-1,-1) to have a low probability: